## StyleGAN2 PyTorch

In [ ]:
# Clone the StyleGAN2-ADA repository and go inside the directory

!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
cd /media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch


# Preparing Custom dataset

1- Download the CXR8 dataset and extract all the images into one folder.

2- All the images are square and the same size.

3- Code needs the dataset to be in .tfrecords format. We need to convert our dataset to this format. 
   StyleGAN2-ADA has made a script that makes this conversion easy.

# Resizing the images

In [ ]:
import cv2
import os

# Set the path where the original folders are located
original_path = '/media/ntu/volume1/home/s123md305_01/Documents/Dataset'

# Set the path where the resized images will be stored
destination_path = '/media/ntu/volume1/home/s123md305_01/Documents/ResizedFull'

# Define the new image size
im_size = 256

# Function to ensure directory exists or create it
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Iterate over all the folders in the original directory
for label_folder in os.listdir(original_path):
    # Make sure it's a folder
    label_path = os.path.join(original_path, label_folder)
    if os.path.isdir(label_path):
        # Create a corresponding folder in the destination directory
        dest_label_path = os.path.join(destination_path, label_folder)
        ensure_dir(dest_label_path)
        
        # Iterate over all images in the current label's folder
        for file in os.listdir(label_path):
            # Check if it's an image
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(label_path, file)
                img = cv2.imread(img_path)  # reading that image as array
                img = cv2.resize(img, (im_size, im_size))  # resizing the image
                print(f'Resized image {file} to {img.shape}')
                
                # Save the resized image to the corresponding label folder in the destination directory
                resized_img_path = os.path.join(dest_label_path, file)
                cv2.imwrite(resized_img_path, img)
                print(f'Saved resized image to {resized_img_path}')


# Converting images into .tfrecords

In [ ]:
import os
import subprocess

# Set the base paths for the resized images and the target TF folders
resized_base_path = '/media/ntu/volume1/home/s123md305_01/Documents/Resized'
tf_base_path = '/media/ntu/volume1/home/s123md305_01/Documents/TF'

# Function to ensure directory exists or create it
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Iterate over both 'resized 256' and 'resized 512' directories
for size_dir in ['Resized 256']:
    size_path = os.path.join(resized_base_path, size_dir)
    
    # Iterate over each label subfolder in the current size directory
    for label_folder in os.listdir(size_path):
        label_path = os.path.join(size_path, label_folder)
        if os.path.isdir(label_path):
            # Define the destination directory within the TF folder structure
            tf_label_path = os.path.join(tf_base_path, size_dir, label_folder)
            ensure_dir(tf_label_path)
            
            # Define the source directory (which is the current label subfolder)
            origin_path = label_path
            
            # Define the destination directory for the TF records
            destination_path = tf_label_path
            
            # Run the dataset_tool.py command
            command = f'python dataset_tool.py --dest="{destination_path}" --source="{origin_path}"'
            subprocess.run(command, shell=True)


In [ ]:
#Training StyleGAN2-ADA

# snap is how often you want to save the model and sample results
# res is what image resolution you want to train on
# augpipe is augmentation pipes, such as 'blit', 'geom', 'color', 'filter', 'noise', 'cutout' or combination of these



import os
import subprocess

# Define the paths
base_data_path = '/media/ntu/volume1/home/s123md305_01/Documents/TF/Remainding'
results_base_path = './results/256'

# Ensure the results directory exists
os.makedirs(results_base_path, exist_ok=True)

# Iterate over each subfolder in the 'Resized 256' directory
for subfolder in sorted(os.listdir(base_data_path)):
    # Construct the full path to the subfolder
    data_path = os.path.join(base_data_path, subfolder)
    if os.path.isdir(data_path):  # Check if it is a directory
        # Construct the output directory path
        outdir = os.path.join(results_base_path, subfolder)

        # Ensure the output directory for this subfolder exists
        os.makedirs(outdir, exist_ok=True)

        # Construct the training command
        command = [
            'python', 'train.py',
            '--outdir', outdir,
            '--data', data_path,
            '--mirror','1',
            '--augpipe', 'bgcfnc',
            '--gpus', '2',
            '--aug','ada',
            '--metrics','none'
            
        ]

        # Print the command to the console for debugging
        print('Running command:', ' '.join(command))

        # Initiate the training process and wait for it to complete
        subprocess.run(command, check=True)


## Generate the images

In [ ]:
!python generate.py --outdir=Generated256/Infiltration --trunc=0.5 --seeds=1-9547 --network=/media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch/results/256/Infiltration/00000-Infiltration-mirror-auto2-ada-bgcfnc/network-snapshot-005000.pkl


In [ ]:
import cv2
import os

# Set the path where the original images are located
original_path = '/media/ntu/volume1/home/s123md305_01/Documents/Dataset'

# Set the path where the resized images will be stored
destination_path = '/media/ntu/volume1/home/s123md305_01/Documents/Resized224'

# Define the new image size
im_size = 224

# Function to ensure the destination directory exists or create it
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Ensure the destination directory exists
ensure_dir(destination_path)

# Iterate over all files in the original directory
for file in os.listdir(original_path):
    # Check if the file is an image
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(original_path, file)
        img = cv2.imread(img_path)  # Read the image as an array
        img = cv2.resize(img, (im_size, im_size))  # Resize the image
        print(f'Resized image {file} to {img.shape}')
        
        # Save the resized image to the destination directory
        resized_img_path = os.path.join(destination_path, file)
        cv2.imwrite(resized_img_path, img)
        print(f'Saved resized image to {resized_img_path}')


In [ ]:
import cv2
import os

# Set the path where the original images are located
original_path = '/media/ntu/volume1/home/s123md305_01/Documents/Dataset'

# Set the path where the resized images will be stored
destination_path = '/media/ntu/volume1/home/s123md305_01/Documents/Resized112'

# Define the new image size
im_size = 112

# Function to ensure the destination directory exists or create it
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Ensure the destination directory exists
ensure_dir(destination_path)

# Iterate over all files in the original directory
for file in os.listdir(original_path):
    # Check if the file is an image
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(original_path, file)
        img = cv2.imread(img_path)  # Read the image as an array
        img = cv2.resize(img, (im_size, im_size))  # Resize the image
        print(f'Resized image {file} to {img.shape}')
        
        # Save the resized image to the destination directory
        resized_img_path = os.path.join(destination_path, file)
        cv2.imwrite(resized_img_path, img)
        print(f'Saved resized image to {resized_img_path}')


In [ ]:
/media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch/Generated256

In [ ]:
import cv2
import os

# Set the path where the original folders are located
original_path = '/media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch/Generated256'

# Define the new image sizes
image_sizes = [224, 128, 112]

# Function to ensure directory exists or create it
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to resize and save an image
def resize_and_save(image_path, destination_path, size):
    img = cv2.imread(image_path)  # Read the image
    img = cv2.resize(img, (size, size))  # Resize the image
    cv2.imwrite(destination_path, img)  # Save the resized image

# Iterate over all the folders in the original directory
for label_folder in os.listdir(original_path):
    label_path = os.path.join(original_path, label_folder)
    if os.path.isdir(label_path):
        # For each target size, create a new root directory and process images
        for size in image_sizes:
            new_root_folder = f'Generated{size}'
            dest_label_path = os.path.join('/media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch', new_root_folder, label_folder)
            ensure_dir(dest_label_path)

            # Iterate over all images in the current label's folder
            for file in os.listdir(label_path):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(label_path, file)
                    resized_img_path = os.path.join(dest_label_path, file)
                    resize_and_save(img_path, resized_img_path, size)
                    print(f'Resized image {file} to {size}x{size} and saved to {resized_img_path}')


In [ ]:
import cv2
import os

# Original and destination paths
original_path = '/media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch/Generated256'
destination_base_path = '/media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch'

# Image sizes for resizing
image_sizes = [224, 128, 112]

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def resize_and_save(image_path, destination_path, size):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (size, size))
        cv2.imwrite(destination_path, img)
    else:
        print(f"Failed to load image {image_path}")

for label_folder in os.listdir(original_path):
    label_path = os.path.join(original_path, label_folder)
    if os.path.isdir(label_path):
        for size in image_sizes:
            # Create a new root folder for each size
            destination_path = os.path.join(destination_base_path, f'Generated{size}', label_folder)
            ensure_dir(destination_path)

            for file in os.listdir(label_path):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(label_path, file)
                    resized_img_path = os.path.join(destination_path, file)
                    resize_and_save(img_path, resized_img_path, size)


In [ ]:
import os

def find_zero_byte_files(start_path):
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_path = os.path.join(root, file)
            if os.path.getsize(file_path) == 0:
                print(file_path)

# Set the root directory you want to start the search from
root_directory = '/media/ntu/volume1/home/s123md305_01/Documents/StyleGAN2-with-adaptive-discriminator-augmentation-ADA--main/stylegan2-ada-pytorch/Generated256'
find_zero_byte_files(root_directory)


In [ ]:
import cv2
import os

# Set the path where the images are located
original_path = '/media/ntu/volume1/home/s123md305_01/Documents/ResizedFull'

# Define the new image sizes
image_sizes = [256, 128]

# Function to ensure directory exists or create it
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to resize and save an image
def resize_and_save(image_path, destination_path, size):
    img = cv2.imread(image_path)  # Read the image
    img = cv2.resize(img, (size, size))  # Resize the image
    cv2.imwrite(destination_path, img)  # Save the resized image

# Ensure the original path contains images
if os.path.isdir(original_path):
    # For each target size, create a new directory for resized images
    for size in image_sizes:
        new_folder_name = f'Resized{size}'
        dest_path = os.path.join('/media/ntu/volume1/home/s123md305_01/Documents', new_folder_name)
        ensure_dir(dest_path)

        # Iterate over all images in the original directory
        for file in os.listdir(original_path):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(original_path, file)
                resized_img_path = os.path.join(dest_path, file)
                resize_and_save(img_path, resized_img_path, size)
                print(f'Resized image {file} to {size}x{size} and saved to {resized_img_path}')


In [ ]:
import os

# The root directory containing sub-subfolders with images
root_dir = '/media/ntu/volume1/home/s123md305_01/Documents/Generated'

# Walk through the directory
for root, dirs, files in os.walk(root_dir):
    for file in files:
        # Check if the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            # The current sub-subfolder name
            sub_subfolder_name = os.path.basename(root)
            
            # New file name with the sub-subfolder name as prefix
            new_file_name = f"{sub_subfolder_name}_{file}"
            
            # Current file path
            current_file_path = os.path.join(root, file)
            
            # New file path
            new_file_path = os.path.join(root, new_file_name)
            
            # Rename the file
            os.rename(current_file_path, new_file_path)
            print(f'Renamed {file} to {new_file_name}')

print('All images have been renamed.')


In [ ]:
import pandas as pd
import os

# The root folder where the separated class folders are located
root_folder = '/media/ntu/volume1/home/s123md305_01/Documents/Generated/Generated256'

# The path for the new CSV file
csv_file_path = '/media/ntu/volume1/home/s123md305_01/Documents/Generated/reconstructed_labels.csv'

# Prepare a list to hold the data
data = []

# Iterate over all folders in the root directory, where each folder's name is a label
for label_folder in os.listdir(root_folder):
    label_path = os.path.join(root_folder, label_folder)
    
    # Check if it's a directory
    if os.path.isdir(label_path):
        # List all files in the directory
        for image_name in os.listdir(label_path):
            # Check if it's a file and not a subdirectory
            if os.path.isfile(os.path.join(label_path, image_name)):
                # Append the image name and label to the data list
                data.append([image_name, label_folder])

# Convert the data list to a DataFrame
df = pd.DataFrame(data, columns=['Image Index', 'Finding Labels'])

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print('CSV file has been created with image names and labels.')


In [ ]:
import os
import shutil

base_path = '/media/ntu/volume1/home/s123md305_01/Documents/Generated'

# Iterate through each subfolder in the base directory
for subfolder in os.listdir(base_path):
    subfolder_path = os.path.join(base_path, subfolder)

    # Check if it's a directory to avoid processing files
    if os.path.isdir(subfolder_path):
        # Construct the new destination subfolder name and path
        new_subfolder_name = 'Com' + subfolder  # Prepend 'Com' to the original subfolder name
        new_subfolder_path = os.path.join(base_path, new_subfolder_name)

        # Create the new subfolder if it doesn't exist
        if not os.path.exists(new_subfolder_path):
            os.makedirs(new_subfolder_path)
            print(f'Created new subfolder: {new_subfolder_path}')

        # Iterate through each sub-subfolder in the current subfolder
        for sub_subfolder in os.listdir(subfolder_path):
            sub_subfolder_path = os.path.join(subfolder_path, sub_subfolder)

            # Again, check if it's a directory
            if os.path.isdir(sub_subfolder_path):
                # Iterate through each file in the sub-subfolder
                for filename in os.listdir(sub_subfolder_path):
                    file_path = os.path.join(sub_subfolder_path, filename)
                    
                    # Construct the destination path in the new corresponding subfolder
                    destination_path = os.path.join(new_subfolder_path, filename)
                    
                    # Check if the file is an image before copying
                    if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                        # Copy the file to the destination
                        shutil.copy(file_path, destination_path)
                        print(f'Copied {file_path} to {destination_path}')
